In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# load the dataset
fraud_df = pd.read_csv("financial_fraud_detection_dataset.csv")

# remove unnamed index column if it exists
fraud_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')

In [77]:
# initial data exploration
fraud_df.head()

,transaction_id,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,fraud_type,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash
0,T100000,2023-08-22T09:22:43.516168,ACC877572,ACC388389,343.78,withdrawal,utilities,Tokyo,mobile,False,NaN,NaN,-0.21,3,0.22,card,13.101.214.112,D8536477
1,T100001,2023-08-04T01:58:02.606711,ACC895667,ACC944962,419.65,withdrawal,online,Toronto,atm,False,NaN,NaN,-0.14,7,0.96,ACH,172.52.47.194,D2622631
2,T100002,2023-05-12T11:39:33.742963,ACC733052,ACC377370,2773.86,deposit,other,London,pos,False,NaN,NaN,-1.78,20,0.89,card,185.98.35.23,D4823498
3,T100003,2023-10-10T06:04:43.195112,ACC996865,ACC344098,1666.22,deposit,online,Sydney,pos,False,NaN,NaN,-0.60,6,0.37,wire_transfer,107.136.36.87,D9961380
4,T100004,2023-09-24T08:09:02.700162,ACC584714,ACC497887,24.43,transfer,utilities,Toronto,mobile,False,NaN,NaN,0.79,13,0.27,ACH,108.161.108.255,D7637601


In [78]:
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 18 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   transaction_id               object 
 1   timestamp                    object 
 2   sender_account               object 
 3   receiver_account             object 
 4   amount                       float64
 5   transaction_type             object 
 6   merchant_category            object 
 7   location                     object 
 8   device_used                  object 
 9   is_fraud                     bool   
 10  fraud_type                   object 
 11  time_since_last_transaction  float64
 12  spending_deviation_score     float64
 13  velocity_score               int64  
 14  geo_anomaly_score            float64
 15  payment_channel              object 
 16  ip_address                   object 
 17  device_hash                  object 
dtypes: bool(1), float64(4), int64(1), object(1

In [79]:
fraud_df.isnull().sum()

transaction_id                       0
timestamp                            0
sender_account                       0
receiver_account                     0
amount                               0
transaction_type                     0
merchant_category                    0
location                             0
device_used                          0
is_fraud                             0
fraud_type                     4820447
time_since_last_transaction     896513
spending_deviation_score             0
velocity_score                       0
geo_anomaly_score                    0
payment_channel                      0
ip_address                           0
device_hash                          0
dtype: int64

In [80]:
# drop columns that are not needed for analysis
fraud_df.drop(columns=['transaction_id', 'sender_account', 'receiver_account', 
                       'fraud_type', 'time_since_last_transaction', 
                       'ip_address', 'device_hash'], inplace=True, errors='ignore')

In [81]:
# check null values after dropping columns
fraud_df.isnull().sum()

timestamp                   0
amount                      0
transaction_type            0
merchant_category           0
location                    0
device_used                 0
is_fraud                    0
spending_deviation_score    0
velocity_score              0
geo_anomaly_score           0
payment_channel             0
dtype: int64

In [82]:
# extract just the date part and convert to datetime
fraud_df['timestamp'] = pd.to_datetime(fraud_df['timestamp'].str.split('T').str[0])

In [83]:
# verify the conversion
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 11 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   timestamp                 datetime64[ns]
 1   amount                    float64       
 2   transaction_type          object        
 3   merchant_category         object        
 4   location                  object        
 5   device_used               object        
 6   is_fraud                  bool          
 7   spending_deviation_score  float64       
 8   velocity_score            int64         
 9   geo_anomaly_score         float64       
 10  payment_channel           object        
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 386.2+ MB


In [84]:
print(fraud_df['timestamp'].head())

0   2023-08-22
1   2023-08-04
2   2023-05-12
3   2023-10-10
4   2023-09-24
Name: timestamp, dtype: datetime64[ns]


In [85]:
# check the current class distribution

print(fraud_df['is_fraud'].value_counts())
print("\nPercentages:")
print(fraud_df['is_fraud'].value_counts(normalize=True) * 100)

is_fraud
False    4820447
True      179553
Name: count, dtype: int64

Percentages:
is_fraud
False    96.40894
True      3.59106
Name: proportion, dtype: float64


In [86]:
# separate fraud and non-fraud cases
fraud_cases = fraud_df[fraud_df['is_fraud'] == True]
non_fraud_cases = fraud_df[fraud_df['is_fraud'] == False]

print(f"\nTotal fraud cases: {len(fraud_cases)}")
print(f"Total non-fraud cases: {len(non_fraud_cases)}")


Total fraud cases: 179553
Total non-fraud cases: 4820447


In [87]:
# calculate target for 1:2 fraud to non-fraud ratio
target_non_fraud = len(fraud_cases) * 2

# define features for stratification
stratify_features = ['transaction_type', 'merchant_category', 'location', 'device_used', 'payment_channel']

In [88]:
# sample non-fraud cases using stratification
sampled_non_fraud, _ = train_test_split(
    non_fraud_cases,
    train_size=target_non_fraud,
    stratify=non_fraud_cases[stratify_features],
    random_state=42
)

In [89]:
print(f"sampled non-fraud cases: {len(sampled_non_fraud)}")

sampled non-fraud cases: 359106


In [90]:
# combine & shuffle fraud cases with sampled non-fraud cases
balanced_df = pd.concat([fraud_cases, sampled_non_fraud], ignore_index=True)
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [91]:
# verify the new distribution
print(balanced_df['is_fraud'].value_counts())
print("\nPercentages:")
print(balanced_df['is_fraud'].value_counts(normalize=True) * 100)
print(f"\nFinal dataset shape: {balanced_df.shape}")

is_fraud
False    359106
True     179553
Name: count, dtype: int64

Percentages:
is_fraud
False    66.666667
True     33.333333
Name: proportion, dtype: float64

Final dataset shape: (538659, 11)


In [92]:
# check for any remaining null values
balanced_df.isnull().sum()

timestamp                   0
amount                      0
transaction_type            0
merchant_category           0
location                    0
device_used                 0
is_fraud                    0
spending_deviation_score    0
velocity_score              0
geo_anomaly_score           0
payment_channel             0
dtype: int64

In [93]:
# save the balanced dataset to CSV
balanced_df.to_csv('cleaned_fraud_dataset.csv', index=False)

print(f"File: 'cleaned_fraud_dataset.csv'")
print(f"Rows: {len(balanced_df)}")
print(f"Columns: {len(balanced_df.columns)}")

File: 'cleaned_fraud_dataset.csv'
Rows: 538659
Columns: 11


In [94]:
# final dataset info
balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538659 entries, 0 to 538658
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   timestamp                 538659 non-null  datetime64[ns]
 1   amount                    538659 non-null  float64       
 2   transaction_type          538659 non-null  object        
 3   merchant_category         538659 non-null  object        
 4   location                  538659 non-null  object        
 5   device_used               538659 non-null  object        
 6   is_fraud                  538659 non-null  bool          
 7   spending_deviation_score  538659 non-null  float64       
 8   velocity_score            538659 non-null  int64         
 9   geo_anomaly_score         538659 non-null  float64       
 10  payment_channel           538659 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(5)
me

In [95]:
balanced_df.head()

,timestamp,amount,transaction_type,merchant_category,location,device_used,is_fraud,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel
0,2023-12-18,28.44,transfer,retail,Singapore,web,False,0.23,11,0.93,wire_transfer
1,2023-02-06,64.88,payment,retail,Toronto,pos,False,0.44,4,0.40,wire_transfer
2,2023-07-26,5.68,transfer,online,Dubai,web,True,0.28,18,0.09,wire_transfer
3,2023-04-27,11.97,transfer,utilities,Toronto,atm,True,-1.31,1,0.63,wire_transfer
4,2023-03-14,191.39,withdrawal,retail,Tokyo,pos,False,1.10,12,0.16,UPI
